In [12]:
import pandas as pd
import folium

In [13]:
traject = pd.read_csv('trajectory_for_every_client_line_with_order_with_altitude.xlsx - Sheet1.csv')

In [14]:
traject.head()

,client_line,current_station,next_station,present in photo,track_1,track_2,Altitude,Coordinates,Unnamed: 8,Unnamed: 9
0,Bilbao-Donostia,Casco Viejo,Zurbaranbarri,1.0,1.0,40.0,17,"43.26018,-2.92217",NaN,NaN
1,Bilbao-Donostia,Zurbaranbarri,Txurdinaga,1.0,2.0,39.0,69,"43.26423,-2.91273",NaN,NaN
2,Bilbao-Donostia,Txurdinaga,Otxarkoaga,1.0,3.0,38.0,67,"43.25798,-2.90677",NaN,NaN
3,Bilbao-Donostia,Otxarkoaga,Kukullaga,1.0,4.0,37.0,90,"43.25949,-2.90010",NaN,NaN
4,Bilbao-Donostia,Kukullaga,Etxebarri,1.0,5.0,36.0,30,"43.25112,-2.88965",NaN,NaN


In [15]:
traject[['latitude', 'longitude']] = traject['Coordinates'].str.split(',', expand=True).astype(float)

In [16]:
plot_data = traject[['client_line', 'current_station', 'latitude', 'longitude']]

In [17]:
plot_data['client_line'].unique()

array(['Bilbao-Donostia', 'Donostia-Hendaia', 'L3 Txori',
       'Lutxana-Sondika', 'Ramal Altza', 'Ramal Amorebieta-Bermeo',
       'Ramal Lasarte'], dtype=object)

In [18]:
plot_data.describe()

,latitude,longitude
count,88.000000,88.000000
mean,43.272772,-2.462918
std,0.057461,0.391260
min,43.165440,-2.941320
25%,43.227105,-2.824078
50%,43.281020,-2.561220
75%,43.311535,-2.016810
max,43.417800,-1.781950


In [28]:
m = folium.Map(location=[43.272488, -2.363647], zoom_start=10
)

# Define a dictionary to map client lines to colors
color_dict = {
    'Bilbao-Donostia': 'blue',
    'Ramal Amorebieta-Bermeo': 'green',
    'Ramal Lasarte': 'red', 
    'Donostia-Hendaia': 'pink',
    'L3 Txori': 'black',
    'Lutxana-Sondika': 'gray',
    'Ramal Altza': 'orange'
    # Add other client lines and their corresponding colors here
}

# Function to choose marker color based on client_line
def get_marker_color(client_line):
    return color_dict.get(client_line)  # Default color is gray

# Add markers to the map
for idx, row in plot_data.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=row['current_station'],
        icon=folium.Icon(color=get_marker_color(row['client_line']))
    ).add_to(m)

# Create a legend as a separate HTML string
legend_html = '''
     <div style="position: fixed; 
                 bottom: 30px; left: 30px; width: 150px; height: 170px; 
                 border:2px solid grey; z-index:9999; font-size:10px;
                 background-color:white;
                 ">
     &nbsp;<b> Client Line Legend </b><br>
     &nbsp;<i class="fa fa-map-marker fa-2x" style="color:blue"></i>&nbsp; Bilbao-Donostia<br>
     &nbsp;<i class="fa fa-map-marker fa-2x" style="color:green"></i>&nbsp; Ramal Amorebieta-Bermeo<br>
     &nbsp;<i class="fa fa-map-marker fa-2x" style="color:red"></i>&nbsp; Ramal Lasarte<br>
     &nbsp;<i class="fa fa-map-marker fa-2x" style="color:pink"></i>&nbsp; Donostia-Hendaia<br>
     &nbsp;<i class="fa fa-map-marker fa-2x" style="color:black"></i>&nbsp; L3 Txori<br>
     &nbsp;<i class="fa fa-map-marker fa-2x" style="color:gray"></i>&nbsp; Lutxana-Sondika<br>
     &nbsp;<i class="fa fa-map-marker fa-2x" style="color:orange"></i>&nbsp; Ramal Altza<br>
     </div>
     '''

m.get_root().html.add_child(folium.Element(legend_html))

# Save the map to an HTML file
m.save('map.html')

# Display the map in a Jupyter Notebook (optional)
m